In [1]:
# Clone the repository and install the necessary packages
!git clone https://github.com/JakubMartinka/karlsruhe2024.git
!pip3 install mlatom
!pip3 install geometric
!pip3 install py3Dmol

fatal: destination path 'karlsruhe2024' already exists and is not an empty directory.


In [2]:
import mlatom as ml

# Universal ANI-1ccx MLP
ANI-1ccx is targeting CCSD(T)/CBS - let's see how well it works.

This is just for demonstration - we recommend to use UAIQM in real applications (https://xacs.xmu.edu.cn/docs/mlatom/tutorial_uaiqm.html)

In [3]:
ani1ccx = ml.models.methods(method='ANI-1ccx')

## Example 1: Heat of formation
Let's calculate the heat of formation of vinylacetylene

In [4]:
# initial xyz geometry
va_init = ml.molecule.from_xyz_string('''8

C             0.00000     0.00000     0.00000
C             1.33718     0.00000     0.00000
C             2.04862     1.23465     0.00000
C             2.65517     2.30065     0.00000
H             3.17326     3.20500     0.00000
H            -0.56880     0.91919     0.00000
H            -0.56652    -0.92292    -0.00000
H             1.95564    -0.90366     0.00000
''')

In [6]:
# step 1 - optimize the geometry
geomopt = ml.simulations.optimize_geometry(model=ani1ccx, initial_molecule=va_init, program='geometric')
va_opt = geomopt.optimized_molecule
va_opt.view()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
# step 2 - run frequency and thermochemistry calculations
va_freq = ml.molecule.from_numpy(va_opt.get_xyz_coordinates(), va_opt.get_atomic_numbers())
ml.simulations.thermochemistry(model=ani1ccx, molecule=va_freq, program='pyscf')

In [9]:
# Check vibration analysis
print("Mode     Frequencies     Reduced masses     Force Constants")
print("           (cm^-1)            (AMU)           (mDyne/A)")
for ii in range(len(va_freq.frequencies)):
    print("%d   %13.4f   %13.4f   %13.4f"%(ii,va_freq.frequencies[ii],va_freq.reduced_masses[ii],va_freq.force_constants[ii]))

# Check thermochemistry results
print('')
print(f"Zero-point vibrational energy: {va_freq.ZPE} Hartree")
print(f"Enthalpy at 298 K: {va_freq.H} Hartree")
print(f"Gibbs Free energy at 298 K: {va_freq.G} Hartree")
print(f"Heat of formation at 298 K: {va_freq.DeltaHf298 * ml.constants.Hartree2kcalpermol} kcal/mol")

Mode     Frequencies     Reduced masses     Force Constants
           (cm^-1)            (AMU)           (mDyne/A)
0        234.9794          3.5415          0.1152
1        305.4731          2.6231          0.1442
2        542.3088          1.4545          0.2520
3        546.6856          1.2320          0.2169
4        559.5900          2.3633          0.4360
5        588.4182          1.5589          0.3180
6        900.9572          1.3032          0.6233
7        966.7358          2.6537          1.4612
8        991.7384          1.2771          0.7401
9       1163.7754          1.4014          1.1183
10       1332.0918          1.4146          1.4789
11       1499.7102          1.2292          1.6289
12       1747.0063          4.0168          7.2230
13       2206.7751          5.7738         16.5663
14       3206.2240          1.0906          6.6056
15       3266.3954          1.0645          6.6914
16       3345.5178          1.1205          7.3890
17       3389.1891         

In [10]:
# check the uncertainty (UQ threshold is 1.68 kcal/mol)
va_freq.ani1ccx.energy_standard_deviation * ml.constants.Hartree2kcalpermol

0.36480208562649497

In [11]:
# let's view the first vibrational normal mode
va_freq.view(normal_mode=0)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Example 2: MD
Let's do MD of the boring hydrogen molecule H2

In [12]:
# use random initial conditions
mol = ml.data.molecule.from_xyz_string('''2

H             0.0000000000000           0.0000000000000           0.0000000000000
H             0.0000000000000           0.0000000000000           0.8000000000000
''')
init_cond_db = ml.generate_initial_conditions(molecule=mol,
                                                generation_method='random',
                                                initial_temperature=300,           # set initial temperature in K,
                                                number_of_initial_conditions=1     # we just want to run a single trajectory
                                             )
init_mol = init_cond_db[0]

In [13]:
# Initializing thermostat
nose_hoover = ml.md.Nose_Hoover_thermostat(temperature=300, molecule=init_mol)

# Run molecular dynamics
dyn = ml.md(model=ani1ccx,
            molecule_with_initial_conditions=init_mol,
            thermostat=nose_hoover,
            ensemble='NVT',
            time_step=0.5, # fs
            maximum_propagation_time=100.0 # fs
           )

# Dump trajectory
traj = dyn.molecular_trajectory
traj.dump(filename='traj', format='plain_text')
traj.dump(filename='traj.h5', format='h5md')

print(f"Number of steps in the trajectory: {len(traj.steps)}")

Number of steps in the trajectory: 201


In [14]:
traj.view()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [15]:
for step in traj.steps:
    print(f'{step.step:3d}  {step.time:8.2f}   {step.molecule.internuclear_distance(0, 1):8.2f} {step.molecule.atoms[1].xyz_coordinates[2] - step.molecule.atoms[0].xyz_coordinates[2]:8.2f}')

  0      0.00       0.80     0.80
  1      0.50       0.79     0.79
  2      1.00       0.76     0.76
  3      1.50       0.68     0.68
  4      2.00       0.58     0.58
  5      2.50       0.48     0.48
  6      3.00       0.41     0.41
  7      3.50       0.36     0.36
  8      4.00       0.33     0.33
  9      4.50       0.30     0.30
 10      5.00       0.28     0.28
 11      5.50       0.26     0.26
 12      6.00       0.24     0.24
 13      6.50       0.23     0.23
 14      7.00       0.22     0.22
 15      7.50       0.20     0.20
 16      8.00       0.19     0.19
 17      8.50       0.19     0.19
 18      9.00       0.18     0.18
 19      9.50       0.17     0.17
 20     10.00       0.16     0.16
 21     10.50       0.16     0.16
 22     11.00       0.15     0.15
 23     11.50       0.15     0.15
 24     12.00       0.14     0.14
 25     12.50       0.14     0.14
 26     13.00       0.13     0.13
 27     13.50       0.13     0.13
 28     14.00       0.13     0.13
 29     14.50 